In [ ]:
from hyperopt import hp,fmin,tpe
import numpy as np
import matplotlib.pyplot as plt
import random
import h5py
import copy
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import tensorflow as tf

In [ ]:
#get data
f=h5py.File('dataset.h5','r')
testsig=list(f['test'])
trainsig=list(f['train'])

In [ ]:
def f(space):
    def generator_f(data,size,snr,X,Y_):
        signal=np.zeros([size,8192])
        label=np.zeros([size,2])
        data=random.sample(data,size/2)
        data=np.array(data)
        for i in range(size):
            if i%2==0:
                r=random.randint(0,1638)
                label[i]=[0,1]
                signal[i,0:8191-r:1]=data[i/2,r:8191:1]*snr
                signal[i]+=np.random.normal(0,1,8192)
                signal[i]=signal[i]/np.std(signal[i])
            else:
                label[i]=[1,0]
                signal[i]=np.random.normal(0,1,8192)
        feed_dict={
            X:signal,
            Y_:label,
        }
        return feed_dict    

    def generator_r(data,size,snr,X,Y_):
        signal=np.zeros([size,8192])
        label=np.zeros([size,2])
        data=random.sample(data,size/2)
        data=np.array(data)
        for i in range(size):
            if i%2==0:
                r=random.randint(0,1638)
                label[i]=[0,1]
                signal[i,0:8191-r:1]=data[i/2,r:8191:1]*random.uniform(snr,2)
                signal[i]+=np.random.normal(0,1,8192)
                signal[i]=signal[i]/np.std(signal[i])
            else:
                label[i]=[1,0]
                signal[i]=np.random.normal(0,1,8192)
        feed_dict={
            X:signal,
            Y_:label,
        }
        return feed_dict    
    
    sess=tf.Session()
    k1=space['k1']
    k2=space['k2']
    k3=space['k3']
    c1=space['c1']
    c2=space['c2']
    c3=space['c3']
    c4=space['c4']
    print(' k1: '+str(k1)+' k2: '+str(k2)+' k3: '+str(k3)+' c1: '+str(c1)+' c2: '+str(c2)+' c3: '+str(c3)+' c4: '+str(c4))
#initial weight
    def weight_variable(name,shape):
        return tf.get_variable(name,shape=shape,initializer=tf.contrib.layers.xavier_initializer())
#intial bias
    def bias_variable(name,shape):
        return tf.get_variable(name,shape=shape,initializer=tf.constant_initializer(0.1))
#placeholder
    with tf.name_scope('Input'):
        X=tf.placeholder(tf.float32,[None,8192])
    with tf.name_scope('Label'):
        Y_=tf.placeholder(tf.float32,[None,2])

#reshape layer
    with tf.name_scope('Reshape_layer'):
        XX=tf.reshape(X,[-1,8192,1,1])
#convolution layer 1
    w_conv1=weight_variable('w_conv1',[k1,1,1,c1])
    b_conv1=bias_variable('b_conv1',[c1])
    with tf.name_scope('Convolution_layer1'):
        conv1=tf.nn.conv2d(XX,w_conv1,strides=[1,1,1,1],padding='SAME')
    print(conv1.shape)

#activation layer 1
    with tf.name_scope('Relu_layer1'):
        h_conv1=tf.nn.relu(conv1+b_conv1)

#pooling layer 1
    with tf.name_scope('Pooling_layer1'):
        h_pool1=tf.nn.max_pool(h_conv1,[1,4,1,1],strides=[1,4,1,1],padding='SAME')
    print(h_pool1.shape)

#convolution layer 2
    w_conv2=weight_variable('w_conv2',[k2,1,c1,c2])
    b_conv2=bias_variable('b_conv2',[c2])
    with tf.name_scope('Convolution_layer2'):
        conv2=tf.nn.atrous_conv2d(h_pool1,w_conv2,rate=4,padding='SAME')
    print(conv2.shape)

#activation layer 2
    with tf.name_scope('Relu_layer2'):
        h_conv2=tf.nn.relu(conv2+b_conv2)

#pooling layer 2
    with tf.name_scope('Pooling_layer2'):
        h_pool2=tf.nn.max_pool(h_conv2,ksize=[1,4,1,1],strides=[1,4,1,1],padding='SAME')
    print(h_pool2)

#convolution layer 3
    w_conv3=weight_variable('w_conv3',[k3,1,c2,c3])
    b_conv3=bias_variable('b_conv3',[c3])
    with tf.name_scope('Convolution_layer3'):
        conv3=tf.nn.atrous_conv2d(h_pool2,w_conv3,rate=4,padding='SAME')
    print(conv3)

#activation layer 3
    with tf.name_scope('Relu_layer3'):
        h_conv3=tf.nn.relu(conv3+b_conv3)

#pooling layer 3
    with tf.name_scope('Pooling_layer3'):
        h_pool3=tf.nn.max_pool(h_conv3,ksize=[1,4,1,1],strides=[1,4,1,1],padding='SAME')
    print(h_pool3.shape)

#flatten layer
    with tf.name_scope('Flatten_layer'):
        h_flatten=tf.reshape(h_pool3,[-1,128*c3])
    print(h_flatten.shape)

#linear layer 1
    w_linear1=weight_variable('w_linear1',[128*c3,c4])
    b_linear1=bias_variable('b_linear1',[c4])

#activation layer 4
    with tf.name_scope('Relu_layer4'):
        h_linear1=tf.nn.relu(tf.matmul(h_flatten,w_linear1)+b_linear1)
    
#linear layer 2
    w_linear2=weight_variable('w_linear2',[c4,2])
    b_linear2=bias_variable('b_linear2',[2])

#activation layer 5
    Ylogits=tf.matmul(h_linear1,w_linear2)+b_linear2
    with tf.name_scope('Output'):
        Y=tf.nn.softmax(Ylogits)

#loss function
    cross_entropy=tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits,labels=Y_)
    cross_entropy=tf.reduce_mean(cross_entropy)

#accuracy
    is_correct=tf.equal(tf.argmax(Y,1),tf.argmax(Y_,1))
    accuracy=tf.reduce_mean(tf.cast(is_correct,tf.float32))*100

#optimization
    optimizer=tf.train.AdamOptimizer(0.001)
    global_step = tf.Variable(0, name='global_step', trainable=False)
    train_step=optimizer.minimize(cross_entropy,global_step=global_step)
#run
    sess.run(tf.global_variables_initializer())
    
    try:
        for i in range(30000):
            snr=0.3
            feed_dict=generator_r(trainsig,50,snr,X,Y_)
            sess.run(train_step,feed_dict=feed_dict)
        test=generator_f(testsig,1480,snr,X,Y_)
        a=sess.run(accuracy,feed_dict=test)
        sess.close()
        tf.reset_default_graph()
        print('accuracy: '+str(a))
        return -a
    except Exception,e:
        sess.close()
        tf.reset_default_graph()
        print e
        return 0

In [ ]:
space={
    'k1':hp.choice('k1',[4,8,16,32]),
    'k2':hp.choice('k2',[4,8,16,32]),
    'k3':hp.choice('k3',[4,8,16,32]),
    'c1':hp.choice('c1',[8,16,32,64,128]),
    'c2':hp.choice('c2',[16,32,64,128,256]),
    'c3':hp.choice('c3',[16,32,64,128,256]),
    'c4':hp.choice('c4',[16,32,64,128,256])
    
}
best=fmin(fn=f,space=space,algo=tpe.suggest,max_evals=50000)
print(best)